<div align="center">

# **F-adjoint concept**





We consider the simple case of a fully-connected deep multi-layer perceptron (MLP) composed of $L$ layers trained in a supervised setting.  We will denote such an architecture by $ A[N_0, \cdots, N_\ell,\cdots, N_L]$ where $N_0$ is the size of the input layer, $N_\ell$ is the size of hidden layer $\ell$,
and $N_L$ is the size of the output layer; $L$ is defined as the depth of the  (MLP).

**Notations**

$$
  \begin{array}{|l|l|}
  \hline\\
  Term & Description \\ \hline
 W^{\ell}\in\mathbb{R}^{N_{\ell}\times N_{\ell-1}} & \mathrm{Weight\ matrix\ of\ the\ layer}\ \ell  \\ \hline
 Y^{\ell}\in\mathbb{R}^{N_{\ell}}  & \mathrm{Preactivation\ vector\ at\ layer}\ {\ell}, Y^{\ell} = W^{\ell}X^{\ell-1} \\ \hline
 X^{\ell}\in\mathbb{R}^{N_{\ell}} & \mathrm{Activition\ vector\ at\ the\ layer} \ {\ell}, X^{\ell} =\sigma^{\ell}(Y^{\ell})\\ \hline
 \sigma^\ell:\  \mathbb{R}^{N_{\ell}}\ni Y^{\ell}\longmapsto\sigma^{\ell}(Y^{\ell})\in\mathbb{R}^{N_{\ell}} & \mathrm{Coordinate-wise\ activition\ function\ of\ the\ layer}\ {\ell}\\ \hline
   \end{array}$$


# The F-propagation and F-adjoint

We   shall recall and revise the  definition of the this notion and provide some straightforward properties and improvements of this adjoint-like representation.



## Definition of an $F$-propagation

Let $X^0\in\mathbb{R}^{N_0}$ be a given data, $\sigma$ be a coordinate-wise map from $\mathbb{R}^{N_\ell}$ into $\mathbb{R}^{N_{\ell}}$ and $W^{\ell}\in \mathbb{R}^{{N_{\ell}}\times{N_{\ell-1}}}$ for all ${1\leq \ell\leq L}$. We say that we have a two-step recursive F-propagation   $F$  through the (MLP) $A[N_0,\cdots, N_L]$ if   one has the following family of vectors
$$
F(X^0):=\begin{Bmatrix}X^{0},Y^{1},X^{1},\cdots,X^{L-1},Y^{L},X^{L}\end{Bmatrix}\  \mathrm{where}\  Y^\ell=W^{\ell}X^{\ell-1}, \ X^\ell=\sigma(Y^\ell),\ \ell=1,\cdots, L.
$$
Before going further, let us point that in the above definition the prefix "F" stands for "Feed-forward".

## Definition of the $F$-adjoint of an $F$-propagation

Let $X^0\in\mathbb{R}^{N_0}$ be a given data and let  $X^L_*\in\mathbb{R}^{N_L}$ be a given vector.  We define the F-adjoint propagation  ${F}_{*}$, through the (MLP) $A[N_0,\cdots, N_L]$, associated to the F-propagation  $F(X^0)$  as follows
$$
F_{*}(X^{0}, X^{L}_{*}):=\begin{Bmatrix}X^{L}_{*}, Y^{L}_{*}, X^{L-1}_{*},\cdots, X^{1}_{*},Y^{1}_{*}, X^{0}_{*} \end{Bmatrix}\  \mathrm{where}\  Y^\ell_{*}=X^{\ell}_{*}\odot {\sigma}'(Y^\ell), \ X^{\ell-1}_{*}=(W^\ell)^\top Y^\ell_{*},\ \ell=L,\cdots, 1.
$$




## Reference

<div id="refs" class="references">


Boughammoura, A. (2023). Backpropagation and F-adjoint. arXiv preprint arXiv:2304.13820.(https://arxiv.org/abs/2304.13820)

</div>




As a consequense, one may rewrite the  algorithm as follows:

1. Require: $x,y$
2. Ensure: $W:=(W^\ell)_{1\leq \ell\leq L}$

   1.  Initialize $W^\ell , \ell=1,\ldots,L$
   2.  Function: $F$-propagation($x,W,\sigma$)
    
        1.  $X^0\leftarrow x$
        2.  $F\leftarrow\{X^0\}$
        3.  For $\ell=1$ to $L$:
                            
            $Y^\ell\leftarrow W^\ell X^{\ell-1}$
                 
            $X^\ell\leftarrow\sigma(Y^{\ell})$
            
            $F\leftarrow Y^\ell,X^\ell$
            
            End For
          
       Return $F$
   

 Function: $F_*$-propagation($F, J, y, \sigma',\eta$)


     $$X_*^L\leftarrow \frac{\partial J}{\partial X^L}(X^L, y)$$

      2. For $\ell= L$ to $1$:                              
         $Y_*^\ell \leftarrow X_*^{\ell}\odot\sigma'(Y^\ell) $
          
           $X^{\ell-1}_{*} \leftarrow ({W^{\ell}})^\top Y^\ell_{*}$
                                
           $F_*\leftarrow Y_*^\ell, X^{\ell-1}_{*}$
           
           End For
           
       Return $F_*$


In [ ]:
import numpy as np
# The activation function and it's derivative
sigma = lambda x: 1/(1+np.exp(-x))
d_sigma= lambda x: sigma(x)*(1-sigma(x))

# The F-propagation function
def F_prop(X, W):
    L=len(W)
    X=X=np.append(X, np.ones((X.shape[0],1)), axis = 1)
    F_X = {'X0':X.T}
    for h in range(1, L+1):
        F_X[f'Y{h}']=np.dot(W[f'W{h}'], F_X[f'X{h-1}'])
        if h!=L:
            F_X[f'X{h}']=np.concatenate((sigma(F_X[f'Y{h}']), np.ones((1,X.shape[0]))), axis = 0)
        else:
            F_X[f'X{h}']=sigma(F_X[f'Y{h}'])
    return F_X

# The F-adjoint propagation function
def F_star_prop(X,y, W):
    L=len(W)
    F_X = F_prop(X, W)
    FX_star={f'X*{L}':(F_X[f'X{L}']-y)}
    Y_star={}
    for h in reversed(range(1, L+1)):
        FX_star[f'Y*{h}']=FX_star[f'X*{h}']*(d_sigma(F_X[f'Y{h}']))
        FX_star[f'X*{h-1}']=np.delete(W[f'W{h}'], -1, axis = 1).T.dot(FX_star[f'Y*{h}'])
    return FX_star

# Update the weights layer-wise by using the F-adjoint propagation: local-learnig approach
def F_star_W(X,y, W,eta=0.5):
    L=len(W)
    F_X = F_prop(X, W)
    FX_star={'X*2':(F_X['X2']-y)}
    Y_star={}
    ##
    FX_star['Y*2']=FX_star['X*2']*(d_sigma(F_X['Y2']))
    W['W2']= W['W2'] -eta*FX_star['Y*2'].dot(F_X['X1'].T)
    ##
    FX_star['X*1']=np.delete(W['W2'], -1, axis = 1).T.dot(FX_star['Y*2'])
    FX_star['Y*1']=FX_star['X*1']*(d_sigma(F_X['Y1']))
    W['W1']= W['W1'] -eta*FX_star['Y*1'].dot(F_X['X0'].T)
    return W

# Update the weights globally by using the F-adjoint propagation: nonlocal-learnig approach
def Grad_star(X,y, W, eta=0.5):
    L=len(W)
    F_X = F_prop(X, W)
    FX_star=F_star_prop(X,y, W)
    Grad={}
    for h in range(1, L+1):
        Grad[f'W{h}']=W[f'W{h}']-eta*FX_star[f'Y*{h}'].dot(F_X[f'X{h-1}'].T)
    return Grad

# F-adjoint propagation and application to local and nonlocal learning

**Compute the $F$ and $F_*$ propagations of the XOR data**

In [ ]:
np.set_printoptions(precision=3)
# Learning the XOR dataset
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([0, 1, 1, 0])
#X = np.array([[0, 0]]); y=np.array([[1]])
W1=np.array([[1, -1, 1], [-1, 1, -1]])
W2=np.array([[1, -1, 1]])
W={'W1':W1, 'W2':W2}

In [ ]:
F_prop(X, W)

{'X0': array([[0., 0., 1., 1.],
        [0., 1., 0., 1.],
        [1., 1., 1., 1.]]),
 'Y1': array([[ 1.,  0.,  2.,  1.],
        [-1.,  0., -2., -1.]]),
 'X1': array([[0.731, 0.5  , 0.881, 0.731],
        [0.269, 0.5  , 0.119, 0.269],
        [1.   , 1.   , 1.   , 1.   ]]),
 'Y2': array([[1.462, 1.   , 1.762, 1.462]]),
 'X2': array([[0.812, 0.731, 0.853, 0.812]])}

In [ ]:
F_star_prop(X,y, W)

{'X*2': array([[ 0.812, -0.269, -0.147,  0.812]]),
 'Y*2': array([[ 0.124, -0.053, -0.018,  0.124]]),
 'X*1': array([[ 0.124, -0.053, -0.018,  0.124],
        [-0.124,  0.053,  0.018, -0.124]]),
 'Y*1': array([[ 0.024, -0.013, -0.002,  0.024],
        [-0.024,  0.013,  0.002, -0.024]]),
 'X*0': array([[ 0.049, -0.026, -0.004,  0.049],
        [-0.049,  0.026,  0.004, -0.049]])}

**Learning the XOR dataset with layer-wise function: F_star_W(X,y, W)**

In [ ]:
# Learning the XOR dataset with layer-wise function: F_star_W(X,y, W)
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([0, 1, 1, 0])
#X = np.array([[0, 0]]); y=np.array([[1]])
W1=np.array([[1, -1, 1], [-1, 1, -1]])
W2=np.array([[1, -1, 1]])
W={'W1':W1, 'W2':W2}
##
X2=F_prop(X, W)['X2']
##
X2_c=np.where(X2>=0.5,1,0)
k=0
while (X2_c -y).any() != 0 :
#for k in range(100):
    #print('X',X)
    X2=F_prop(X, W)['X2']
    X2_c=np.where(X2>=0.5,1,0)
    FX_star=F_star_prop(X,y, W)


    print('X2',X2,k+1)
    W=F_star_W(X,y, W)
    #W=Grad_star(X,y, W)
    X=X-FX_star['X*0'].T
    k+=1
    print("================================")

X2 [[0.812 0.731 0.853 0.812]] 1
X2 [[0.781 0.707 0.833 0.78 ]] 2
X2 [[0.746 0.681 0.81  0.743]] 3
X2 [[0.706 0.654 0.784 0.7  ]] 4
X2 [[0.662 0.627 0.758 0.654]] 5
X2 [[0.618 0.602 0.733 0.607]] 6
X2 [[0.575 0.582 0.712 0.561]] 7
X2 [[0.535 0.567 0.695 0.518]] 8
X2 [[0.499 0.558 0.684 0.48 ]] 9


Let us notes that the number of itairations for this learning task is 9.


**Learning the XOR dataset with global gradient function:Grad_star(X,y, W)**

In [ ]:
# Learning the XOR dataset with global gradient function:Grad_star(X,y, W)
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([0, 1, 1, 0])
#X = np.array([[0, 0]]); y=np.array([[1]])*np.sqrt(6)
W1=np.array([[1, -1, 1], [-1, 1, 0]])*(20)
W2=np.array([[1, -1, 1]])
W={'W1':W1, 'W2':W2}
##
X2=F_prop(X, W)['X2']
##
X2_c=np.where(X2>=0.5,1,0)
k=0
while (X2_c -y).any() != 0 :
#for k in range(10):
    #print('X',X)
    X2=F_prop(X, W)['X2']
    X2_c=np.where(X2>=0.5,1,0)
    FX_star=F_star_prop(X,y, W)


    print('X2=',X2,k+1)
    #W=F_star_W(X,y, W)
    W=Grad_star(X,y, W)
    X=X-FX_star['X*0'].T##(X0-Xstar0) as input
    k+=1
    print("================================")

X2= [[0.818 0.622 0.881 0.818]] 1
X2= [[0.481 0.713 0.862 0.481]] 2


Let us notes that the number of itairations for this learning task is 2.